In [39]:
from dashboardUtils import *
from pySheets import DictArray

In [40]:
## normalize spacenames in both datasets
spacesDA = DictArray(pd.read_excel('spacestmp.xls'))
for i,space in enumerate(spacesDA):
    spacesDA[i]['name'] = space['name'].replace('  ',' ').strip()
    
membersDA = DictArray(pd.read_excel('memberstmp.xls'))
for i,member in enumerate(membersDA):
    thisGuysSpaces = eval(member['spaces'])
    thisGuysSpaces = [space.replace('  ',' ').strip() for space in thisGuysSpaces]
    membersDA[i]['spaces'] = thisGuysSpaces

In [42]:
spaceNamesFromSpacesDA = {space['name'] for space in spacesDA}

spaceNamesFromMembers = set()
for member in membersDA:
    for space in member['spaces']:
        spaceNamesFromMembers.add(space)
        
        
#sanity check
print(len(spaceNamesFromSpacesDA), len(spaceNamesFromMembers))
spaceNamesFromSpacesDA.difference(spaceNamesFromMembers)
# BUG for some reason "e-NABLE Web Central" is missing from spacesDA
# good enough for now

48 47


{'e-NABLE Web Central'}

In [51]:
#Connections 
connections = []
for row in membersDA:
    label  = row['name']
    for space in row['spaces']:
        connections.append(OrderedDict(From=row['name'], To=space))
        

In [53]:
#create spaces dict
spaceDict={}
for row in spacesDA:
    spaceDict[row['name']] = row['href']

In [54]:
def spacesWithLinks(memberSpaces):
    lines=[]
    for name in memberSpaces:
        lines.append(f'[{name}]({spaceDict[name]})') #markdown
    return('\n'.join(lines))

#print(spacesWithLinks(membersDA[0]['spaces']))

In [70]:
elements = []
for member in membersDA:
    Label = member['name']
    URL =   member['href']
    Description = spacesWithLinks(member['spaces'])
    elements.append(OrderedDict( Label = Label, URL = URL, Description = Description, Type='Person') )

In [71]:
#add spaces to the elements
for space in spacesDA:
    Label = space['name']
    URL   = space['href']
    elements.append(OrderedDict( Label = Label, URL=URL, Type='Space'))

In [73]:
from pySheets import get_DFB, write_DFB


In [74]:
GSURL = 'https://docs.google.com/spreadsheets/d/1ks7Ge5FJh85OlS_VVMIWO1BD1ju3R3zcAOGEPxhz1m4/edit#gid=440006482'

In [76]:
DFB = OrderedDict(Elements=DataFrame(elements), Connections = DataFrame(connections))
write_DFB(DFB, GSURL )